In [52]:
import cv2
from matplotlib import pyplot as plt
from matplotlib import image as img
import numpy as np
import os
import imutils
from fastai.vision import *
from PIL import Image
import pandas as pd

In [53]:
path = os.getcwd()
img_path = path+'/data/h5'
path_lbl = path+'/data/h5/masks2/'
path_infer_400 = img_path+'/roc/01'

get_mask_fn = lambda x, suffix: img_path+'/roc/'+f'{suffix}/{x.stem}.tif'
get_gt_fn = lambda x: path_lbl+f'/{x.stem}_mask.tif'
get_stem = lambda x: {x.stem}

il = ImageList.from_folder(path_infer_400)

In [54]:
def weird_division(n, d):
    return n / d if d else 0


def process_image(filename, roc, area=5):
    
    im = cv2.imread(get_mask_fn(filename, roc), 0)
    gt = cv2.imread(get_gt_fn(filename), 0)
    im_contours = cv2.findContours(im, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    im_contours = imutils.grab_contours(im_contours)
    gt_contours = cv2.findContours(gt, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    gt_contours = imutils.grab_contours(gt_contours)
    tot_inmask, tot_extra= 0,0
    #found_cnt=set()
    
    for pred_cnt in im_contours:
        M = cv2.moments(pred_cnt)
        if  M['m00'] < area:

            cX = 0 if M["m00"] == 0 else int(M["m10"] / M["m00"])
            cY = 0 if M["m00"] == 0 else int(M["m01"] / M["m00"])

            belongs = False
            for mask_cnt in gt_contours:
                if cv2.pointPolygonTest(mask_cnt, (cX, cY), False) > 0:
                    belongs = True
                    #found_cnt.add(str(mask_cnt))
                    break
            if belongs:
                tot_inmask += 1
            else:
                tot_extra += 1
                
    stem = get_stem(filename).pop()
    accuracy = weird_division(tot_inmask, tot_inmask+tot_extra)
    n_in = tot_inmask
    n_out = tot_extra
    n_gt = len(gt_contours)
    
    return([stem, accuracy, n_in, n_out, n_gt])
    

In [55]:
process_image(il.items[1], "L10")

['8493200_5774800', 0.09815950920245399, 16, 147, 567]

In [64]:
results = []
for idx, im in enumerate(il):
    results.append(process_image(il.items[idx], "L10"))
    
results_L10 = pd.DataFrame(results, columns =['Loc ID', 'Accuracy', 'n_in', 'n_out', 'n_gt']) 
results_L10.to_csv('L10_result.csv')

In [79]:
# results = []
# for idx, im in enumerate(il):
#     results.append(process_image(il.items[idx], "L15"))
# results_L15 = pd.DataFrame(results, columns =['Loc ID', 'Accuracy', 'n_in', 'n_out', 'n_gt']) 
# results_L15.to_csv('L15_result.csv')
# results = []
# for idx, im in enumerate(il):
#     results.append(process_image(il.items[idx], "L20"))
# results_L20 = pd.DataFrame(results, columns =['Loc ID', 'Accuracy', 'n_in', 'n_out', 'n_gt']) 
# results_L20.to_csv('L20_result.csv')
# results = []
# for idx, im in enumerate(il):
#     results.append(process_image(il.items[idx], "L25"))
# results_L25 = pd.DataFrame(results, columns =['Loc ID', 'Accuracy', 'n_in', 'n_out', 'n_gt']) 
# results_L25.to_csv('L25_result.csv')
# results = []
# for idx, im in enumerate(il):
#     results.append(process_image(il.items[idx], "L30"))
# results_L30 = pd.DataFrame(results, columns =['Loc ID', 'Accuracy', 'n_in', 'n_out', 'n_gt']) 
# results_L30.to_csv('L30_result.csv')
# results = []
# for idx, im in enumerate(il):
#     results.append(process_image(il.items[idx], "L35"))
# results_L35 = pd.DataFrame(results, columns =['Loc ID', 'Accuracy', 'n_in', 'n_out', 'n_gt']) 
# results_L35.to_csv('L35_result.csv')
results = []
for idx, im in enumerate(il):
    results.append(process_image(il.items[idx], "L40"))
results_L40 = pd.DataFrame(results, columns =['Loc ID', 'Accuracy', 'n_in', 'n_out', 'n_gt']) 
results_L40.to_csv('L40_result.csv')
results = []
for idx, im in enumerate(il):
    results.append(process_image(il.items[idx], "L45"))
results_L45 = pd.DataFrame(results, columns =['Loc ID', 'Accuracy', 'n_in', 'n_out', 'n_gt']) 
results_L45.to_csv('L45_result.csv')
results = []
for idx, im in enumerate(il):
    results.append(process_image(il.items[idx], "L50"))
results_L50 = pd.DataFrame(results, columns =['Loc ID', 'Accuracy', 'n_in', 'n_out', 'n_gt']) 
results_L50.to_csv('L50_result.csv')

In [80]:
frames = [results_L10, results_L15, results_L20,results_L25,results_L30,results_L35, results_L40, results_L45, results_L50]

In [81]:
df_cat = pd.concat(frames, keys=['L10', 'L15', 'L20', 'L25','L30', 'L35', 'L40','L45','L50'])

In [82]:
df_cat

Loc ID  Accuracy  n_in  n_out  n_gt
L10 0     9061200_5690800  0.000000     0      3     1
    1     8493200_5774800  0.098160    16    147   567
    2     9377200_5525600  0.000000     0      4    11
    3     9298000_5556400  0.000000     0      4    17
    4     8492000_5775200  0.250000     2      6    11
...                   ...       ...   ...    ...   ...
L50 1782  9367600_5522000  0.166667     3     15    14
    1783  9298400_5562400  0.000000     0      2    29
    1784  9061200_5681200  0.000000     0     10    87
    1785  9156800_5818400  0.000000     0      0     1
    1786  9366400_5515600  0.068966     2     27   104

[16083 rows x 5 columns]

In [83]:
df_cat.to_csv('merged_blob_result.csv')

In [84]:
df_cat.reset_index().groupby("level_0").mean()

,level_1,Accuracy,n_in,n_out,n_gt
level_0,,,,,
L10,893.0,0.087973,9.631226,77.284275,136.744264
L15,893.0,0.091892,8.950196,65.578064,136.744264
L20,893.0,0.095088,8.659205,59.145495,136.744264
L25,893.0,0.095495,8.115837,54.555680,136.744264
L30,893.0,0.101187,7.984891,50.034135,136.744264
L35,893.0,0.104026,7.348629,44.926693,136.744264
L40,893.0,0.110612,6.831561,39.863458,136.744264
L45,893.0,0.115246,6.668719,35.924454,136.744264
L50,893.0,0.122584,6.405708,33.006156,136.744264


In [86]:
df_cat.to_pickle('picklename')

In [87]:
df_cat = pd.read_pickle('picklename')